In [1]:
import mmi.mmi_client
import mmi.tracker as tr
import numpy as np

from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

from IPython.display import display
from ipywidgets import interactive

import threading

DEBUG:shapely.geos:Found GEOS DLL: <CDLL '/opt/conda/lib/python3.6/site-packages/shapely/.libs/libgeos_c-09576097.so.1.9.0', handle 1018a70 at 0x7f8228bdbef0>, using it.
DEBUG:shapely.geos:Trying `CDLL(libc.so.6)`
DEBUG:shapely.geos:Library path: 'libc.so.6'
DEBUG:shapely.geos:DLL: <CDLL 'libc.so.6', handle 7f824ef7f4e0 at 0x7f824d292c88>


In [2]:
time = []

data = dict(
    H = [],
    zb = [],
    zs = [],
    xz = [],
)


def get_data(model, t):
    
    global time, data
    
    time.append(t)
    slider.children[0].max = len(time) - 1

    for var in data.keys():
        data[var].append(model.get_var(var)[0])

                
def listen(model):
    
    global data
    
    t_old = model.get_current_time()
    while model.get_current_time() <= model.get_end_time():
        t_new = model.get_current_time()
        if (t_old != t_new):
            
            get_data(model, t_new)
                
        t_old = t_new
    time.sleep(1)
    
    
def update_plot(t):
    
    global time, data
    
    try:
        p.title.text = ('t = %0.2f s' % time[t])
        waterline.data_source.data['y'] = data['zs'][t] + data['H'][t]
    except:
        p.title.text = ('t = %0.2f s, does not exist yet' % time[t])
        
    push_notebook()

In [3]:
model = mmi.mmi_client.MMIClient('tcp://xbeach-orchestrator:50010')
slider = interactive(update_plot, t=(0, len(time)+1, 1 ))

In [4]:
get_data(model, 0)

In [5]:
output_notebook()

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
p = figure(tools=TOOLS, 
           width=700, 
           height=350, 
           title='t = ??? s', 
           x_axis_label='distance in x-direction [m]', 
           y_axis_label='height [m+NAP]')

bath = p.line(data['xz'][0], data['zb'][0], line_width=2, color='black')
waterline = p.line(data['xz'][0], data['zs'][0] + data['H'][0], color='blue')

hover = HoverTool(renderers=[waterline])

hover.tooltips = [
    ('water level', '$y')
]
p.add_tools(hover)

show(p, notebook_handle=True)

Loading BokehJS ...

In [6]:
thread = threading.Thread(target=lambda: listen(model))
display(slider)
thread.start()